In [1]:
!pip install soundfile

In [2]:
from google.colab import drive
import os  
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
!ls

drive  sample_data


In [4]:
resources_path= "drive/MyDrive/project_SpeechT/"

In [5]:
os.listdir(resources_path)

['argentinian', 'chilean', 'columbian', 'venezuelan', 'peruvian']

Creating CSV files for all the data of my dataset 


In [6]:
import os 
import csv 
import re 


splitted= []
speakID= []
audioId= []
transcription= []
temp=[]
lang_sex_info= []
speakerID= []

def from_txt_to_csv (file_txt, csv_file_name): 
  with open(file_txt, 'r', encoding= "utf-8") as in_file:
    stripped = (line.strip() for line in in_file)
    for line in stripped: 
      speakID.append((re.findall (r"(.*?)_", line)))
      splitted.append(line.split())
    for e in speakID:
      lang_sex_info.append(e[0])
      speakerID.append((e[0]+ "_" + e[1]))
    for e in splitted:
      audioId.append(e[0])
      temp.append(e[1:])
  for i in temp: 
    transcription.extend([" ".join(i)])
    
  with open(csv_file_name, 'w') as out_file:
    writer = csv.writer(out_file)
    writer.writerow(('Language and sex info', 'SpeakerId','RecordId', 'Transcription'))
    writer.writerows(zip(lang_sex_info,speakerID,audioId, transcription))

    
    

In [7]:
#from_txt_to_csv("drive/MyDrive/project_SpeechT/venezuelan/vem_data.txt", "drive/MyDrive/project_SpeechT/venezuelan/vem_data.csv")

In [8]:
%cd /content/drive/MyDrive/project_SpeechT/

/content/drive/MyDrive/project_SpeechT


In [9]:
!ls

argentinian  chilean  columbian  peruvian  venezuelan


Creating a list with all the files from each Spanish accent folder

In [11]:
import glob 
all_files_list= []
for audiofile in glob.glob("*/*/*.wav"):
  all_files_list.append(audiofile)

In [12]:
all_files_list

['argentinian/es_ar_male/arm_06136_00545342835.wav',
 'argentinian/es_ar_male/arm_06136_00486481946.wav',
 'argentinian/es_ar_male/arm_06136_00527044669.wav',
 'argentinian/es_ar_male/arm_06136_00095815237.wav',
 'argentinian/es_ar_male/arm_06136_00386857099.wav',
 'argentinian/es_ar_male/arm_06136_00035963132.wav',
 'argentinian/es_ar_male/arm_06136_00383702325.wav',
 'argentinian/es_ar_male/arm_06136_00279667527.wav',
 'argentinian/es_ar_male/arm_06136_00389029506.wav',
 'argentinian/es_ar_male/arm_06136_00358528761.wav',
 'argentinian/es_ar_male/arm_06136_00062303749.wav',
 'argentinian/es_ar_male/arm_06136_00260165323.wav',
 'argentinian/es_ar_male/arm_06136_00136487766.wav',
 'argentinian/es_ar_male/arm_06136_00243767965.wav',
 'argentinian/es_ar_male/arm_06136_00367218284.wav',
 'argentinian/es_ar_male/arm_06136_00035496108.wav',
 'argentinian/es_ar_male/arm_06136_00537994622.wav',
 'argentinian/es_ar_male/arm_06136_00040030944.wav',
 'argentinian/es_ar_male/arm_06136_00459698934

Total of wav files in my dataset 

In [13]:
print(len(all_files_list))

22988


In [14]:
target= ["es_ar_female", "es_ar_male", "es_co_female", "es_co_male", "es_pe_female", "es_pe_male", "es_ve_female", "es_ve_male", "es_cl_female","es_cl_male"]

Dividing te files per target group (target is the label for each accent - female or male) 

In [15]:
#files per target group
word_list=[[] for i in range(len(target))]
for f in all_files_list:
  w = f.split('/')[1]
  word_list[target.index(w)].append(f)

Randomizing the files 

In [16]:
#creating subset
import numpy as np
for i in range(len(word_list)):
  rand_idx = np.random.choice(len(word_list[i]), size=22988) ##Check if the random numbers are with replacement or not
  _tmp = [word_list[i][r] for r in rand_idx]
  word_list[i]=_tmp

Dividing the data for training (70%), validation(15%) and testing (15%)

In [17]:
#partitions 
partitions={}
partitions["train"]=[]
partitions["valid"]=[]
partitions["test"]=[]
for i in range(len(word_list)):
  train_pct = int(len(word_list[i])*0.7)
  valid_pct = int((len(word_list[i])-train_pct)*0.5)
  partitions["train"].extend(word_list[i][:train_pct])
  partitions["valid"].extend(word_list[i][train_pct:train_pct+valid_pct])
  partitions["test"].extend(word_list[i][train_pct+valid_pct:])

print(len(all_files_list),len(partitions["train"]),len(partitions["valid"]),len(partitions["test"]))

22988 160910 34480 34490


In [18]:
print(partitions["train"][:20])

['argentinian/es_ar_female/arf_00295_00291970405.wav', 'argentinian/es_ar_female/arf_07505_00705197188.wav', 'argentinian/es_ar_female/arf_08421_01924979695.wav', 'argentinian/es_ar_female/arf_03349_01198968496.wav', 'argentinian/es_ar_female/arf_09799_01949903395.wav', 'argentinian/es_ar_female/arf_02121_01547085182.wav', 'argentinian/es_ar_female/arf_03397_01964425971.wav', 'argentinian/es_ar_female/arf_02121_00731937686.wav', 'argentinian/es_ar_female/arf_03397_01665251952.wav', 'argentinian/es_ar_female/arf_03349_01985858380.wav', 'argentinian/es_ar_female/arf_03349_01228386144.wav', 'argentinian/es_ar_female/arf_09334_00069195292.wav', 'argentinian/es_ar_female/arf_04766_00387982309.wav', 'argentinian/es_ar_female/arf_08421_01839080183.wav', 'argentinian/es_ar_female/arf_09697_01440312960.wav', 'argentinian/es_ar_female/arf_01523_02138647868.wav', 'argentinian/es_ar_female/arf_03349_01107030345.wav', 'argentinian/es_ar_female/arf_05679_01913572082.wav', 'argentinian/es_ar_female/a

In [19]:
print(partitions["valid"][:20])

['argentinian/es_ar_female/arf_02121_01225250090.wav', 'argentinian/es_ar_female/arf_05679_01566494321.wav', 'argentinian/es_ar_female/arf_02121_02137315718.wav', 'argentinian/es_ar_female/arf_06136_00589185217.wav', 'argentinian/es_ar_female/arf_09334_00626552771.wav', 'argentinian/es_ar_female/arf_01523_00451900903.wav', 'argentinian/es_ar_female/arf_01523_01681361602.wav', 'argentinian/es_ar_female/arf_03853_01654864763.wav', 'argentinian/es_ar_female/arf_05679_00565272725.wav', 'argentinian/es_ar_female/arf_04766_01085934274.wav', 'argentinian/es_ar_female/arf_00295_00549740623.wav', 'argentinian/es_ar_female/arf_06592_00667596930.wav', 'argentinian/es_ar_female/arf_09334_01849340453.wav', 'argentinian/es_ar_female/arf_07049_00111322743.wav', 'argentinian/es_ar_female/arf_08421_00147724197.wav', 'argentinian/es_ar_female/arf_09697_00602298717.wav', 'argentinian/es_ar_female/arf_03398_01596421591.wav', 'argentinian/es_ar_female/arf_08886_00679484666.wav', 'argentinian/es_ar_female/a

Dataloader


In [21]:
 import torch 
from torch.utils import data 
import librosa 
import numpy 
from scipy import signal 

class SASA_data(data.Dataset): 
  def __init__(self, partition, target, max_len=64000, win_len=0.02):                #setting properties of dataloader
    self.max_len = max_len 
    self.win_len = win_len 
    self.eps = np.finfo(np.float64).eps
    self.partition = partition 
    self.target = target

  def __len__(self): 
    return len(self.partition)           #return how big datset is 

  def __getitem__(self,index):          
    audiofile = self.partition[index]      #loading this audiofile
    word = audiofile.split("/")[0]         #label
    label = self.target.index(word) if word in target else len(self.target)-1                   #label for that  file 

    x,sr= librosa.load(audiofile,sr=None)
    x= x[:self.max_len]                                    #cutting the file lenght to 4 sec
    if len(x) < self.max_len:
      xx=np.pad(x, [(0,self.max_len-x.shape[0]),],mode="constant")
      x=xx
    n_fft= int(self.win_len*sr)
    filters= librosa.filters.mel(sr, n_fft, n_mels= 40)
    window= signal.hamming(int(n_fft), sym=False)
    spectrogram= np.abs(librosa.stft(y= x+self.eps, n_fft= n_fft, win_length= n_fft, hop_length= n_fft//2, center= True, window= window))
    melspectrum= np.log(np.dot(filters, spectrogram)+self.eps)
    return torch.from_numpy(melspectrum), label


In [24]:
#trying to check 
d= SASA_data(partitions["train"], target)
f,l = d.__getitem__(16000)
print(f.shape)
print(l) 

torch.Size([40, 134])
tensor([[ -7.4794,  -6.4681,  -6.9111,  ...,  -3.2267,  -3.1995,  -3.1432],
        [ -7.8669,  -7.4351,  -7.7961,  ...,  -1.6406,  -1.6599,  -1.7445],
        [ -9.4604,  -8.8974,  -9.4599,  ...,  -3.0363,  -3.0322,  -2.3986],
        ...,
        [-10.4753, -10.5297, -10.5446,  ...,  -9.0185,  -8.8221,  -8.6305],
        [-10.6042, -10.5866, -10.5133,  ...,  -8.5857,  -8.5024,  -8.6582],
        [-10.7286, -10.6276, -10.5276,  ...,  -8.9749,  -8.9239,  -8.9026]],
       dtype=torch.float64)
9


Model for training 